# QEIR Framework - Custom Analysis Examples

This notebook demonstrates how to create custom analyses using the QEIR framework, including:

1. **Custom Data Integration**
2. **Alternative Model Specifications**
3. **Custom Visualization and Reporting**
4. **Integration with External Tools**
5. **Publication-Ready Output Generation**

In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from qeir.core.hypothesis_testing import QEHypothesisTester, HypothesisTestingConfig
from qeir.utils.hypothesis_data_collector import HypothesisDataCollector
from qeir.utils.data_structures import HypothesisData
from qeir.visualization.publication_visualization import PublicationVisualizationSuite
from qeir.utils.latex_table_generator import LaTeXTableGenerator

plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (14, 10)

print("Custom Analysis Examples - Setup Complete")

## 1. Custom Data Integration

In [ ]:
# Example: Creating custom HypothesisData with your own data
def create_custom_hypothesis_data(start_date='2008-01-01', end_date='2023-12-31'):
    """
    Create custom HypothesisData object with synthetic or external data
    """
    # Create date range
    dates = pd.date_range(start=start_date, end=end_date, freq='M')
    n_obs = len(dates)
    
    # Generate synthetic data (replace with your actual data)
    np.random.seed(42)  # For reproducibility
    
    # Hypothesis 1 data
    central_bank_reaction = pd.Series(
        1000 + 500 * np.random.randn(n_obs).cumsum() + 2000 * (dates.year >= 2008),
        index=dates, name='central_bank_reaction'
    )
    
    confidence_effects = pd.Series(
        80 + 10 * np.random.randn(n_obs) - 20 * (dates.year >= 2008) * (dates.year <= 2010),
        index=dates, name='confidence_effects'
    )
    
    debt_service_burden = pd.Series(
        200 + 50 * np.random.randn(n_obs) + 100 * (dates.year >= 2008),
        index=dates, name='debt_service_burden'
    )
    
    long_term_yields = pd.Series(
        3.0 + 1.5 * np.random.randn(n_obs) - 2.0 * (dates.year >= 2008),
        index=dates, name='long_term_yields'
    )
    
    # Hypothesis 2 data
    qe_intensity = pd.Series(
        0.1 + 0.05 * np.random.randn(n_obs) + 0.3 * (dates.year >= 2008),
        index=dates, name='qe_intensity'
    )
    
    private_investment = pd.Series(
        1000 + 100 * np.random.randn(n_obs) - 200 * (dates.year >= 2008) * (dates.year <= 2010),
        index=dates, name='private_investment'
    )
    
    market_distortions = pd.Series(
        2.0 + 0.5 * np.random.randn(n_obs) + 1.0 * (dates.year >= 2008),
        index=dates, name='market_distortions'
    )
    
    interest_rate_channel = pd.Series(
        4.0 + 1.0 * np.random.randn(n_obs) - 3.0 * (dates.year >= 2008),
        index=dates, name='interest_rate_channel'
    )
    
    # Hypothesis 3 data
    foreign_bond_holdings = pd.Series(
        5000 + 500 * np.random.randn(n_obs) + 1000 * (dates.year >= 2008),
        index=dates, name='foreign_bond_holdings'
    )
    
    exchange_rate = pd.Series(
        100 + 10 * np.random.randn(n_obs) + 5 * (dates.year >= 2008),
        index=dates, name='exchange_rate'
    )
    
    inflation_measures = pd.Series(
        2.0 + 0.5 * np.random.randn(n_obs) - 1.0 * (dates.year >= 2008) * (dates.year <= 2010),
        index=dates, name='inflation_measures'
    )
    
    capital_flows = pd.Series(
        100 + 50 * np.random.randn(n_obs) - 100 * (dates.year >= 2008) * (dates.year <= 2010),
        index=dates, name='capital_flows'
    )
    
    # Create HypothesisData object
    custom_data = HypothesisData()
    custom_data.central_bank_reaction = central_bank_reaction
    custom_data.confidence_effects = confidence_effects
    custom_data.debt_service_burden = debt_service_burden
    custom_data.long_term_yields = long_term_yields
    custom_data.qe_intensity = qe_intensity
    custom_data.private_investment = private_investment
    custom_data.market_distortions = market_distortions
    custom_data.interest_rate_channel = interest_rate_channel
    custom_data.foreign_bond_holdings = foreign_bond_holdings
    custom_data.exchange_rate = exchange_rate
    custom_data.inflation_measures = inflation_measures
    custom_data.capital_flows = capital_flows
    custom_data.dates = dates
    
    custom_data.metadata = {
        'source': 'custom_synthetic_data',
        'creation_timestamp': datetime.now().isoformat(),
        'start_date': start_date,
        'end_date': end_date,
        'observations': n_obs,
        'frequency': 'monthly'
    }
    
    return custom_data

# Create custom data
custom_data = create_custom_hypothesis_data()
print(f"Created custom data with {len(custom_data.dates)} observations")
print(f"Date range: {custom_data.dates.min()} to {custom_data.dates.max()}")

## 2. Custom Analysis with Synthetic Data

In [ ]:
# Run analysis with custom data
config = HypothesisTestingConfig(
    bootstrap_iterations=100,  # Reduced for demonstration
    enable_robustness_tests=False
)

# Initialize tester (no data collector needed for custom data)
tester = QEHypothesisTester(config=config)

# Test all hypotheses with custom data
print("Testing hypotheses with custom data...")

h1_results = tester.test_hypothesis1(custom_data)
h2_results = tester.test_hypothesis2(custom_data)
h3_results = tester.test_hypothesis3(custom_data)

# Display results
print("\nCustom Data Analysis Results:")
print("=" * 40)

for i, (name, results) in enumerate([
    ('Hypothesis 1', h1_results),
    ('Hypothesis 2', h2_results),
    ('Hypothesis 3', h3_results)
], 1):
    print(f"\n{name}:")
    if 'error' not in results.main_result:
        print(f"  ✓ Analysis completed")
        print(f"  Observations: {results.data_period.get('observations', 'N/A')}")
        
        # Hypothesis-specific results
        if i == 1:
            threshold_detected = results.main_result.get('threshold_detected', False)
            print(f"  Threshold detected: {threshold_detected}")
            if threshold_detected:
                print(f"  Threshold value: {results.main_result.get('threshold_value', 'N/A'):.4f}")
        elif i == 2:
            investment_response = results.main_result.get('investment_response_detected', False)
            print(f"  Investment response: {investment_response}")
        elif i == 3:
            spillover_effects = results.main_result.get('spillover_effects_detected', False)
            print(f"  Spillover effects: {spillover_effects}")
    else:
        print(f"  ✗ Analysis failed: {results.main_result['error']}")

## 3. Custom Visualization

In [ ]:
# Create custom visualization dashboard
def create_custom_dashboard(data, results_dict):
    """
    Create a comprehensive dashboard for all three hypotheses
    """
    fig = plt.figure(figsize=(20, 16))
    
    # Create grid layout
    gs = fig.add_gridspec(4, 3, hspace=0.3, wspace=0.3)
    
    # Row 1: Key variables for each hypothesis
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.plot(data.dates, data.long_term_yields, 'b-', linewidth=2)
    ax1.set_title('H1: Long-term Yields', fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.plot(data.dates, data.private_investment, 'g-', linewidth=2)
    ax2.set_title('H2: Private Investment', fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    ax3 = fig.add_subplot(gs[0, 2])
    ax3.plot(data.dates, data.exchange_rate, 'r-', linewidth=2)
    ax3.set_title('H3: Exchange Rate', fontweight='bold')
    ax3.grid(True, alpha=0.3)
    
    # Row 2: Threshold analysis (H1)
    ax4 = fig.add_subplot(gs[1, :])
    
    h1_results = results_dict.get('Hypothesis 1')
    if h1_results and 'error' not in h1_results.main_result:
        threshold_detected = h1_results.main_result.get('threshold_detected', False)
        
        # Scatter plot: threshold variable vs dependent variable
        ax4.scatter(data.debt_service_burden, data.long_term_yields, alpha=0.6, s=50)
        
        if threshold_detected:
            threshold_value = h1_results.main_result.get('threshold_value')
            ax4.axvline(threshold_value, color='red', linestyle='--', linewidth=2, 
                       label=f'Threshold: {threshold_value:.3f}')
            ax4.legend()
        
        ax4.set_xlabel('Debt Service Burden')
        ax4.set_ylabel('Long-term Yields')
        ax4.set_title('H1: Threshold Analysis', fontweight='bold')
    else:
        ax4.text(0.5, 0.5, 'H1 Analysis Failed', ha='center', va='center', 
                transform=ax4.transAxes, fontsize=14)
    
    ax4.grid(True, alpha=0.3)
    
    # Row 3: Investment dynamics (H2)
    ax5 = fig.add_subplot(gs[2, 0])
    ax5.plot(data.dates, data.qe_intensity, 'orange', linewidth=2, label='QE Intensity')
    ax5.set_title('H2: QE Intensity', fontweight='bold')
    ax5.grid(True, alpha=0.3)
    
    ax6 = fig.add_subplot(gs[2, 1])
    ax6.plot(data.dates, data.market_distortions, 'purple', linewidth=2, label='Market Distortions')
    ax6.set_title('H2: Market Distortions', fontweight='bold')
    ax6.grid(True, alpha=0.3)
    
    ax7 = fig.add_subplot(gs[2, 2])
    ax7.plot(data.dates, data.interest_rate_channel, 'brown', linewidth=2, label='Interest Rate Channel')
    ax7.set_title('H2: Interest Rate Channel', fontweight='bold')
    ax7.grid(True, alpha=0.3)
    
    # Row 4: International effects (H3)
    ax8 = fig.add_subplot(gs[3, 0])
    ax8.plot(data.dates, data.foreign_bond_holdings, 'cyan', linewidth=2)
    ax8.set_title('H3: Foreign Bond Holdings', fontweight='bold')
    ax8.grid(True, alpha=0.3)
    
    ax9 = fig.add_subplot(gs[3, 1])
    ax9.plot(data.dates, data.inflation_measures, 'magenta', linewidth=2)
    ax9.set_title('H3: Inflation Measures', fontweight='bold')
    ax9.grid(True, alpha=0.3)
    
    ax10 = fig.add_subplot(gs[3, 2])
    ax10.plot(data.dates, data.capital_flows, 'gray', linewidth=2)
    ax10.set_title('H3: Capital Flows', fontweight='bold')
    ax10.grid(True, alpha=0.3)
    
    plt.suptitle('QEIR Framework - Custom Analysis Dashboard', 
                 fontsize=20, fontweight='bold', y=0.98)
    
    return fig

# Create dashboard
results_dict = {
    'Hypothesis 1': h1_results,
    'Hypothesis 2': h2_results,
    'Hypothesis 3': h3_results
}

dashboard_fig = create_custom_dashboard(custom_data, results_dict)
plt.show()

print("Custom dashboard created successfully")

## 4. Custom Results Summary

In [ ]:
# Create comprehensive results summary
def create_results_summary(results_dict, data_metadata=None):
    """
    Create a comprehensive summary of all hypothesis test results
    """
    summary = {
        'analysis_timestamp': datetime.now().isoformat(),
        'data_info': data_metadata or {},
        'hypothesis_results': {},
        'overall_assessment': {}
    }
    
    successful_tests = 0
    total_tests = len(results_dict)
    
    for hyp_name, results in results_dict.items():
        hyp_summary = {
            'status': 'success' if 'error' not in results.main_result else 'failed',
            'observations': results.data_period.get('observations', 0),
            'period': f"{results.data_period.get('start_date')} to {results.data_period.get('end_date')}"
        }
        
        if 'error' not in results.main_result:
            successful_tests += 1
            
            # Extract key findings based on hypothesis
            if 'Hypothesis 1' in hyp_name:
                hyp_summary['threshold_detected'] = results.main_result.get('threshold_detected', False)
                if hyp_summary['threshold_detected']:
                    hyp_summary['threshold_value'] = results.main_result.get('threshold_value')
                    hyp_summary['statistical_significance'] = results.statistical_significance.get('structural_break_significant', False)
            
            elif 'Hypothesis 2' in hyp_name:
                hyp_summary['investment_response'] = results.main_result.get('investment_response_detected', False)
                hyp_summary['local_projections_fitted'] = results.main_result.get('local_projections_fitted', False)
                hyp_summary['iv_fitted'] = results.main_result.get('iv_fitted', False)
            
            elif 'Hypothesis 3' in hyp_name:
                hyp_summary['spillover_effects'] = results.main_result.get('spillover_effects_detected', False)
                hyp_summary['variables_analyzed'] = results.main_result.get('variables_analyzed', 0)
                
                # Add correlation information if available
                if results.statistical_significance:
                    hyp_summary['correlations'] = {
                        'foreign_exchange': results.statistical_significance.get('foreign_exchange_correlation'),
                        'inflation_exchange': results.statistical_significance.get('inflation_exchange_correlation')
                    }
        else:
            hyp_summary['error'] = results.main_result['error']
        
        summary['hypothesis_results'][hyp_name] = hyp_summary
    
    # Overall assessment
    summary['overall_assessment'] = {
        'success_rate': successful_tests / total_tests,
        'successful_tests': successful_tests,
        'total_tests': total_tests,
        'data_quality': 'synthetic' if data_metadata and data_metadata.get('source') == 'custom_synthetic_data' else 'real'
    }
    
    return summary

# Create results summary
results_summary = create_results_summary(results_dict, custom_data.metadata)

# Display summary
print("COMPREHENSIVE RESULTS SUMMARY")
print("=" * 50)

print(f"\nAnalysis Timestamp: {results_summary['analysis_timestamp']}")
print(f"Data Source: {results_summary['data_info'].get('source', 'Unknown')}")
print(f"Success Rate: {results_summary['overall_assessment']['success_rate']:.1%}")
print(f"Successful Tests: {results_summary['overall_assessment']['successful_tests']}/{results_summary['overall_assessment']['total_tests']}")

print("\nHypothesis Results:")
print("-" * 30)

for hyp_name, hyp_results in results_summary['hypothesis_results'].items():
    print(f"\n{hyp_name}:")
    print(f"  Status: {hyp_results['status'].upper()}")
    print(f"  Observations: {hyp_results['observations']}")
    print(f"  Period: {hyp_results['period']}")
    
    if hyp_results['status'] == 'success':
        if 'threshold_detected' in hyp_results:
            print(f"  Threshold Detected: {hyp_results['threshold_detected']}")
            if hyp_results['threshold_detected']:
                print(f"  Threshold Value: {hyp_results['threshold_value']:.4f}")
                print(f"  Statistically Significant: {hyp_results.get('statistical_significance', 'N/A')}")
        
        if 'investment_response' in hyp_results:
            print(f"  Investment Response: {hyp_results['investment_response']}")
            print(f"  Local Projections Fitted: {hyp_results['local_projections_fitted']}")
        
        if 'spillover_effects' in hyp_results:
            print(f"  Spillover Effects: {hyp_results['spillover_effects']}")
            if 'correlations' in hyp_results:
                corrs = hyp_results['correlations']
                print(f"  Foreign-Exchange Correlation: {corrs.get('foreign_exchange', 'N/A')}")
    else:
        print(f"  Error: {hyp_results.get('error', 'Unknown error')}")

print("\n" + "=" * 50)

## 5. Export Results for External Use

In [ ]:
# Export results to various formats
import json
import os

# Create output directory
output_dir = "custom_analysis_outputs"
os.makedirs(output_dir, exist_ok=True)

# 1. Export summary to JSON
def export_to_json(data, filename):
    """Export data to JSON with proper serialization"""
    def json_serializer(obj):
        if isinstance(obj, (pd.Timestamp, pd.DatetimeIndex)):
            return obj.isoformat()
        elif isinstance(obj, pd.Series):
            return obj.to_dict()
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif hasattr(obj, '__dict__'):
            return obj.__dict__
        return str(obj)
    
    with open(filename, 'w') as f:
        json.dump(data, f, indent=2, default=json_serializer)

# Export results summary
export_to_json(results_summary, f"{output_dir}/results_summary.json")
print(f"✓ Results summary exported to {output_dir}/results_summary.json")

# 2. Export data to CSV
data_df = pd.DataFrame({
    'date': custom_data.dates,
    'central_bank_reaction': custom_data.central_bank_reaction,
    'confidence_effects': custom_data.confidence_effects,
    'debt_service_burden': custom_data.debt_service_burden,
    'long_term_yields': custom_data.long_term_yields,
    'qe_intensity': custom_data.qe_intensity,
    'private_investment': custom_data.private_investment,
    'market_distortions': custom_data.market_distortions,
    'interest_rate_channel': custom_data.interest_rate_channel,
    'foreign_bond_holdings': custom_data.foreign_bond_holdings,
    'exchange_rate': custom_data.exchange_rate,
    'inflation_measures': custom_data.inflation_measures,
    'capital_flows': custom_data.capital_flows
})

data_df.to_csv(f"{output_dir}/hypothesis_data.csv", index=False)
print(f"✓ Data exported to {output_dir}/hypothesis_data.csv")

# 3. Save dashboard figure
dashboard_fig.savefig(f"{output_dir}/analysis_dashboard.png", 
                     dpi=300, bbox_inches='tight', facecolor='white')
print(f"✓ Dashboard saved to {output_dir}/analysis_dashboard.png")

# 4. Create summary report
def create_text_report(results_summary, filename):
    """Create a human-readable text report"""
    with open(filename, 'w') as f:
        f.write("QEIR FRAMEWORK - CUSTOM ANALYSIS REPORT\n")
        f.write("=" * 50 + "\n\n")
        
        f.write(f"Analysis Date: {results_summary['analysis_timestamp']}\n")
        f.write(f"Data Source: {results_summary['data_info'].get('source', 'Unknown')}\n")
        f.write(f"Success Rate: {results_summary['overall_assessment']['success_rate']:.1%}\n\n")
        
        f.write("HYPOTHESIS TEST RESULTS\n")
        f.write("-" * 30 + "\n\n")
        
        for hyp_name, hyp_results in results_summary['hypothesis_results'].items():
            f.write(f"{hyp_name}:\n")
            f.write(f"  Status: {hyp_results['status'].upper()}\n")
            f.write(f"  Observations: {hyp_results['observations']}\n")
            f.write(f"  Period: {hyp_results['period']}\n")
            
            if hyp_results['status'] == 'success':
                if 'threshold_detected' in hyp_results:
                    f.write(f"  Key Finding: Threshold {'detected' if hyp_results['threshold_detected'] else 'not detected'}\n")
                elif 'investment_response' in hyp_results:
                    f.write(f"  Key Finding: Investment response {'detected' if hyp_results['investment_response'] else 'not detected'}\n")
                elif 'spillover_effects' in hyp_results:
                    f.write(f"  Key Finding: Spillover effects {'detected' if hyp_results['spillover_effects'] else 'not detected'}\n")
            else:
                f.write(f"  Error: {hyp_results.get('error', 'Unknown error')}\n")
            
            f.write("\n")
        
        f.write("\nEND OF REPORT\n")

create_text_report(results_summary, f"{output_dir}/analysis_report.txt")
print(f"✓ Text report saved to {output_dir}/analysis_report.txt")

print(f"\nAll outputs saved to '{output_dir}/' directory")
print("Files created:")
for file in os.listdir(output_dir):
    print(f"  - {file}")

## Summary

This notebook demonstrated advanced customization techniques for the QEIR framework:

### Key Techniques Covered:

1. **Custom Data Integration:**
   - Creating `HypothesisData` objects with your own data
   - Synthetic data generation for testing
   - Integration with external data sources

2. **Custom Analysis Workflows:**
   - Running hypothesis tests with custom data
   - Handling analysis results programmatically
   - Error handling and validation

3. **Advanced Visualization:**
   - Multi-panel dashboard creation
   - Custom plotting functions
   - Publication-quality figure generation

4. **Results Processing:**
   - Comprehensive results summarization
   - Structured data extraction
   - Cross-hypothesis comparison

5. **Export and Integration:**
   - JSON export for programmatic use
   - CSV export for spreadsheet analysis
   - High-resolution figure export
   - Human-readable report generation

### Applications:

- **Research Projects:** Integrate with your own datasets
- **Policy Analysis:** Custom scenarios and simulations
- **Academic Papers:** Publication-ready outputs
- **Presentations:** Dashboard-style visualizations
- **Further Analysis:** Export data for other tools

### Next Steps:

1. Replace synthetic data with your actual datasets
2. Customize visualizations for your specific needs
3. Integrate with your existing analysis workflows
4. Use exported results in other tools (R, Stata, Excel)
5. Combine with the other notebooks for comprehensive analysis

This approach allows you to leverage the QEIR framework's robust econometric methods while maintaining full control over your data and analysis workflow.